<a href="https://colab.research.google.com/github/ganesh7111/sales-performance-analytics-python/blob/main/Sales_Peformance_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
import pandas as pd
import random
from google.colab import files

# --- 1. GENERATE 800 REPS (Unique IDs) ---
print("Generating 800 Reps...")

first_names = [
    'James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda',
    'David', 'Elizabeth', 'William', 'Barbara', 'Richard', 'Susan', 'Joseph',
    'Jessica', 'Thomas', 'Sarah', 'Charles', 'Karen', 'Christopher', 'Nancy',
    'Daniel', 'Lisa', 'Matthew', 'Betty', 'Anthony', 'Margaret', 'Mark', 'Sandra',
    'Donald', 'Ashley', 'Steven', 'Kimberly', 'Paul', 'Emily', 'Andrew', 'Donna',
    'Joshua', 'Michelle', 'Kenneth', 'Carol', 'Kevin', 'Amanda', 'Brian', 'Melissa'
]

reps = []
rep_ids = list(range(1001, 1801)) # IDs from 1001 to 1800

for rep_id in rep_ids:
    name = random.choice(first_names)
    # kept targets reasonable
    target = random.choice([40000, 45000, 50000, 55000, 60000, 75000, 80000])
    reps.append([rep_id, name, target])

df_reps = pd.DataFrame(reps, columns=['Rep_ID', 'Name', 'Target_Revenue'])
df_reps.to_excel('reps_800.xlsx', index=False)


# --- 2. GENERATE TRANSACTIONS (High Volume to hit Targets) ---
print("Generating transactions for 800 Reps (this may take a moment)...")

products = ['Laptop', 'Mouse', 'Monitor', 'Keyboard', 'Headset', 'Webcam', 'Docking Station']
regions = ['North', 'South', 'East', 'West']
dates = pd.date_range(start='2023-01-01', end='2023-12-31').strftime('%Y-%m-%d').tolist()

transactions = []

for rep_id in rep_ids:
    # CHANGE 1: Increase sales volume significantly (30 to 70 sales per rep)
    # This ensures totals will range from ~30k to ~150k
    num_sales = random.randint(30, 70)

    for _ in range(num_sales):
        date = random.choice(dates)
        product = random.choice(products)
        region = random.choice(regions)

        # CHANGE 2: Increased prices so targets are easier to hit
        if product == 'Laptop':
            amount = random.randint(1500, 3500) # Laptops are now more expensive
        elif product == 'Monitor':
            amount = random.randint(300, 800)
        elif product == 'Docking Station':
            amount = random.randint(150, 400)
        else:
            amount = random.randint(50, 200)

        transactions.append([date, rep_id, product, amount, region])

df_trans_large = pd.DataFrame(transactions, columns=['Date', 'Rep_ID', 'Product', 'Amount', 'Region'])
df_trans_large.to_excel('transactions_800.xlsx', index=False)

print(f"✅ Created 'reps_800.xlsx' ({len(df_reps)} rows)")
print(f"✅ Created 'transactions_800.xlsx' ({len(df_trans_large)} rows)")

# --- 3. AUTO-DOWNLOAD ---
files.download('reps_800.xlsx')
files.download('transactions_800.xlsx')

Generating 800 Reps...
Generating transactions for 800 Reps (this may take a moment)...
✅ Created 'reps_800.xlsx' (800 rows)
✅ Created 'transactions_800.xlsx' (40455 rows)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Step 1: Install the missing library
!pip install xlsxwriter

# Step 2: Import libraries
import pandas as pd
import numpy as np
from google.colab import files
import xlsxwriter

# --- Step 3: LOAD YOUR FILES HERE ---
# FIX: Just use the filename. Colab looks in the current folder by default.
reps_file_path = 'reps_800.xlsx'
trans_file_path = 'transactions_800.xlsx' # FIX: Pointed this to the correct transaction file

print(f"Reading data from: {reps_file_path} and {trans_file_path}...")
df_targets = pd.read_excel(reps_file_path)
df_trans = pd.read_excel(trans_file_path)

# --- Step 4: Run the Analysis ---

# Aggregation: Sum transaction amounts by Rep ID
df_total_sales = df_trans.groupby('Rep_ID')['Amount'].sum().reset_index()
df_total_sales.rename(columns={'Amount': 'Actual_Sales'}, inplace=True)

# Merge: Combine targets with actual sales
df_final = pd.merge(df_targets, df_total_sales, on='Rep_ID', how='left')
df_final['Actual_Sales'] = df_final['Actual_Sales'].fillna(0)

# Calculations
df_final['Achievement_%'] = (df_final['Actual_Sales'] / df_final['Target_Revenue']) * 100

# Commission Logic: 5% if target met, 2% if not
df_final['Commission'] = np.where(
    df_final['Actual_Sales'] >= df_final['Target_Revenue'],
    df_final['Actual_Sales'] * 0.05,
    df_final['Actual_Sales'] * 0.02
)

# --- Step 5: Create Styled Excel Report ---

output_filename = 'Final_Sales_Report.xlsx'
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')
df_final.to_excel(writer, sheet_name='Summary', index=False)

workbook  = writer.book
worksheet = writer.sheets['Summary']
last_row = len(df_final) + 1

# Define Formats
green_fmt = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100', 'border': 1})
red_fmt   = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006', 'border': 1})
yellow_fmt = workbook.add_format({'bg_color': '#FFEB9C', 'font_color': '#9C5700', 'border': 1})
money_fmt = workbook.add_format({'num_format': '$#,##0', 'border': 1})

# Set Column Widths
worksheet.set_column('A:F', 22)

# Apply Currency Format to Columns C, D, and F
worksheet.set_column('C:D', 22, money_fmt)
worksheet.set_column('F:F', 22, money_fmt)

# --- APPLY CONDITIONAL FORMATTING ---

# 1. Apply Yellow to Columns A-D (Rep_ID to Actual_Sales)
worksheet.conditional_format(f'A2:D{last_row}', {
    'type': 'no_errors',
    'format': yellow_fmt
})

# 2. Apply Yellow to Column F (Commission)
worksheet.conditional_format(f'F2:F{last_row}', {
    'type': 'no_errors',
    'format': yellow_fmt
})

# 3. Apply RED/GREEN to Column E (Achievement)
worksheet.conditional_format(f'E2:E{last_row}', {
    'type': 'cell', 'criteria': '>=', 'value': 100, 'format': green_fmt
})

worksheet.conditional_format(f'E2:E{last_row}', {
    'type': 'cell', 'criteria': '<', 'value': 100, 'format': red_fmt
})

writer.close()
print(f"✅ Report generated: {output_filename}")

# Auto-Download
files.download(output_filename)

Reading data from: reps_800.xlsx and transactions_800.xlsx...
✅ Report generated: Final_Sales_Report.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>